In [ ]:
# import csv
# import os
# for i in range(12,1200):
#     fname = os.path.join('New_Dataset',str(i)+'.csv')
#     with open(fname,'w',newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(["Image","Annotations"])

In [ ]:
def add_on_csv(idx,Image,GTname):
    for i in idx:
        fname = os.path.join('New_Dataset',str(i)+'.csv')
        with open(fname,'a+',newline='') as file:
            writer = csv.writer(file)
            writer.writerow([Image,GTname])

In [ ]:
from __future__ import print_function, division
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from PIL import Image
import torchvision.transforms.functional as TF

In [ ]:
concepts = ["color","object","part","material","scene","texture"]

class conceptLoader(Dataset):
    """Broaden Dataset Image Loader"""
    
    def __init__(self, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            conceptType (string): type of the concept to load.
        """
        self.c_flag = False  # Concept Flag
        csv_file = os.path.join(root_dir,"index.csv") 
        root_dir = os.path.join(root_dir,"images") 
        self.fileNames = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.fileNames)
    

    def load_concept(self,idx,conceptType):
        c_index = str(self.fileNames.iloc[idx, concepts.index(conceptType)+6])
        image_name = str(self.fileNames.iloc[idx, 0])
        if c_index=='nan':
            self.c_flag = False
            # sample = torch.zeros([1, 113, 113])
            sample =0
        else:
            self.c_flag = True
            if conceptType =="scene" or conceptType =="texture":
                sample  = c_index.split(";") 
            elif conceptType =="part":
                items  = c_index.split(";") 
                sample = []
                c_index=[]
                for i in items:
                    img_name = os.path.join(self.root_dir,i)
                    image = Image.open(img_name)
                    px = np.array(image)
                    item = self.decodeClassMask(px)
                    sample.append(item)
                    c_index.append(i)
                
            else:
                img_name = os.path.join(self.root_dir,c_index)
                image = Image.open(img_name)
                px = np.array(image)
                sample = self.decodeClassMask(px)
                #sample = Image.fromarray(sample)
                #sample = TF.to_tensor(sample)     

        # sample.unsqueeze_(0)
        return self.c_flag, sample,image_name,c_index

    def decodeClassMask(self,im):
        ''' Decodes pixel-level object/part class and instance data from
        the given image, previously encoded into RGB channels.'''
    # Classes are a combination of RG channels
        return (im[:,:,0] + (im[:,:,1])*256)



In [ ]:
    dataset_path='./broden1_227'
    # data_obj = imageLoader(dataset_path)
    data_obj = conceptLoader(dataset_path)
    for j in range(63305):
        flag,aa,ImgName, ConName = data_obj.load_concept(j,"material")
        print(j)
        if flag:
            k=np.unique(aa)
            add_on_csv(k,ImgName,ConName)

In [ ]:
    dataset_path='./broden1_227'
    # data_obj = imageLoader(dataset_path)
    data_obj = conceptLoader(dataset_path)
    for j in range(63305):
        flag,aa,ImgName, ConName = data_obj.load_concept(j,"part")

        if flag:
            for i in range(len(aa)):
                k=np.unique(aa[i])
                add_on_csv(k,ImgName,ConName[i])
        print(j)

In [12]:
import numpy as np
aa=np.zeros((2,2))
aa[0,0]=1
aa[aa!=1]=0

print(aa)

[[1. 1.]
 [1. 1.]]


In [2]:
import torch 
x=torch.ones([10,3,155,155])

In [ ]:
y=torch.ones([x.shape[0]])

In [5]:
import os
cwd = os.getcwd()  
dir = os.path.join(cwd,"Tk")
if not os.path.exists(dir):
    os.mkdir(dir)